In [2]:
import numpy as np

---
## Initial Parameters

For a single-period model and $n$ risky assets, we have prices $S_1,\ S_2, ...,\ S_n$, and their respective returns $K_1, K_2,...,K_n$. 

We begin with their expected returns $\mu_i = \mathbf{E}(K_i)$ arranged into a one-row matrix $m = (\mu_1, \mu_2,...,\mu_n)$, standard deviations $\sigma_1, \sigma_2,...,\sigma_n$, and correlations $\rho$.

In [3]:
exp_returns = [0.15, -0.1, 0.1]
st_dev = [0.2, 0.15, 0.1]
corr = [0.3, -0.1, 0.2]

---
## Required Functions for Matrices

For $n$ risky securities, we have $c_{ij} = \textnormal{cov}(K_i, K_j)$ as covariances. They form a covariance matrix:

$$
\mathbf{C} =
\begin{pmatrix}
c_{11} & c_{12} & \cdots & c_{1n} \\
c_{21} & c_{22} & \cdots & c_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
c_{n1} & c_{n2} & \cdots & c_{nn}
\end{pmatrix}
$$

We assume that this matrix is invertible: $\textnormal{det}\mathbf{C}\neq 0$.

Property of covariance 1: $c_{ij} = c_{ji}$, therefore, for a 3x3 matrix, $c_{12}=c_{21},\ c_{23}=c_{32},\ c_{13}=c_{31}$.

Property of covariance 2: $\textnormal{var}(i)=c_{ii}=\sigma_i^2$

With correlations $\rho_{ij} = \frac{c_{ij}}{\sigma_i \sigma_j}$, we can calculate for $c_{ij}$ with $c_{ij}=\rho_{ij}\sigma_i \sigma_j$.

Thus, for three assets, we expect our covariance matrix to appear as

$$
\mathbf{C} =

\begin{pmatrix}
c_{11} & c_{12} & c_{13} \\
c_{21} & c_{22} & c_{23} \\
c_{31} & c_{32} & c_{33}
\end{pmatrix}

=

\begin{pmatrix}
\sigma_{1}^2 & c_{12} & c_{13} \\
c_{21} & \sigma_{2}^2 & c_{23} \\
c_{31} & c_{32} & \sigma_{3}^2
\end{pmatrix}
$$

Below are functions to create a covariance matrix, and its inversion.

In [4]:
def find_covariance_matrix(correlation, standard_deviation):
    n = len(standard_deviation)
    covariance_matrix = np.zeros((n, n))

    # For ONLY THREE assets, correlations based on REQUIRED ORDER [ρ12, ρ23, ρ13] of matrix shape 3x3.
    correlation_index = {(0, 1): 0,
                         (1, 2): 1,
                         (0, 2): 2}
    
    for i in range(n):
        for j in range(i + 1, n):
            rho = correlation[correlation_index[(i, j)] if (i, j) in correlation_index else correlation_index[(j, i)]]
            covariance_matrix[i, j] = rho * standard_deviation[i] * standard_deviation[j]
            covariance_matrix[j, i] = covariance_matrix[i, j]
    
    for i in range(n):
        covariance_matrix[i, i] = standard_deviation[i] ** 2
    
    return covariance_matrix

def invert_matrix(matrix):
    return np.linalg.inv(matrix)

### Find Covariance Matrix and Inverse

Using the functions above, we can compute the required covariance matrix and its inverse.

In [5]:
cov_mat = find_covariance_matrix(corr, st_dev)
cov_mat_inv = invert_matrix(cov_mat)

print(cov_mat, "\n\n", cov_mat_inv)

[[ 0.04    0.009   0.004 ]
 [ 0.009   0.0225 -0.0015]
 [ 0.004  -0.0015  0.01  ]] 

 [[ 29.18632075 -12.57861635 -13.56132075]
 [-12.57861635  50.31446541  12.57861635]
 [-13.56132075  12.57861635 107.31132075]]


---
## Find Weights of Minimum Variance Portfolio (MVP)

#### Standing assumptions:

$w = (w_1, ..., w_n)$: The weight vector.

$m = (\mu_1, ..., \mu_n)$: The return vector.

$u = (1,1,...,1)$

$\mathbf{C}=$ The covariance matrix.

$\mathbf{C}^{-1}=$ The inverse covariance matrix.

Thus, we can use the following formula to compute the weights for the MVP:

$$
\mathbf{w}_{\textnormal{MVP}} = \frac{u\mathbf{C}^{-1}}{u\mathbf{C}^{-1}u^{\mathbf{T}}}
$$

Note: $^{\mathbf{T}}$ denotes matrix transposition.

In [6]:
u = np.ones(len(exp_returns))
mvp_weights = []

u_C_inv = u @ cov_mat_inv
u_C_inv_uT = u_C_inv @ u.T

# u_C_inv is a 1x3 array, whereas u_C_inv_uT is a number (usually float). 
# To achieve weights, we must iterate through the values in the array and divide by u_C_inv_uT.

for i in u_C_inv:
    mvp_weights.append(i / u_C_inv_uT)

print(mvp_weights)

[0.01907692307692311, 0.31507692307692303, 0.6658461538461538]


### Compute MVP Variance and Return

MVP variance $\sigma_{\textnormal{MVP}}^2$ is calculated with: 
$$
\sigma_{\textnormal{MVP}}^2 = \mathbf{w}_{\textnormal{MVP}} \ \cdot\ \mathbf{C}\ \cdot\ \mathbf{w}_{\textnormal{MVP}}^{\mathbf{T}}
$$

MVP return $\mu_{\textnormal{MVP}}$ is simply calculated with:
$$
\mu_{\textnormal{MVP}} = \mathbf{w}_{\textnormal{MVP}} \ \cdot\ m^{\mathbf{T}}
$$

In [7]:
mvp_variance = mvp_weights @ cov_mat @ np.array(mvp_weights).T
mvp_return = mvp_weights @ np.array(exp_returns).T

print(mvp_variance)
print(mvp_return)

0.006262153846153846
0.03793846153846154


---
## Return Constraints

Now that we have tackled finding the minimum variance portfolio and its associated metrics, we can attempt finding the same MVP with a given return constraint. In this example, we will use a return constraint of 10% (In plain English, this can be described as: "Given a required rate of return of 10%, what portfolio would provide the lowest risk?").

From my notes, the weights of this portfolio $w_{\mu}$ can be found by:
$$
w_\mu=\frac{\lambda_1}{2}m\mathbf{C}^{-1}+\frac{\lambda_2}{2}u\mathbf{C}^{-1}
$$
And $\lambda_1,\lambda_2$ are given by:
$$
\begin{bmatrix} 
\lambda_1\\ 
\lambda_2\\ 
\end{bmatrix} = 2 \begin{bmatrix} 
m\mathbf{C}^{-1}m^\mathbf{T} & u\mathbf{C}^{-1}m^\mathbf{T}\\
m\mathbf{C}^{-1}u^\mathbf{T} & u\mathbf{C}^{-1}u^\mathbf{T}\\ 
\end{bmatrix}^{-1}\begin{bmatrix} 
\mu\\ 
1\\ 
\end{bmatrix}
$$

With a lot of work, we can rearrange this to:
$$
\lambda_2 = \frac{2\mu m\mathbf{C}^{-1}u^\mathbf{T}-2m\mathbf{C}^{-1}m^\mathbf{T}}{(m\mathbf{C}^{-1}u^\mathbf{T})^2 - u\mathbf{C}^{-1}u^\mathbf{T}m\mathbf{C}^{-1}m^\mathbf{T}}
$$

$$
\lambda_1 = \frac{2\mu - \lambda_2 u\mathbf{C}^{-1}m^\mathbf{T}}{m\mathbf{C}^{-1}m^\mathbf{T}}
$$

In [8]:
# Simplify by defining variables.
m_C_inv = np.array(exp_returns) @ cov_mat_inv
m_C_inv_mT = m_C_inv @ np.array(exp_returns)
m_C_inv_uT = m_C_inv @ u.T
u_C_inv_uT = u @ cov_mat_inv @ u.T
u_C_inv_mT = u @ cov_mat_inv @ np.array(exp_returns).T

ret_cons = 0.1

lambda_2 = ((2 * ret_cons * m_C_inv_uT) - 2 * m_C_inv_mT) / ((m_C_inv_uT ** 2) - (u_C_inv_uT * m_C_inv_mT))
lambda_1 = ((2 * ret_cons) - (lambda_2 * u_C_inv_mT)) / m_C_inv_mT

print(lambda_2)
print(lambda_1)

0.009789755807027994
0.07207861822513403


Now we use:

$$
w_\mu=\frac{\lambda_1}{2}m\mathbf{C}^{-1}+\frac{\lambda_2}{2}u\mathbf{C}^{-1}
$$

In [9]:
ret_cons_weights = ((lambda_1 / 2) * (np.array(exp_returns) @ cov_mat_inv)) + ((lambda_2 / 2) * (u @ cov_mat_inv))
print(ret_cons_weights)

[0.1691483  0.04228708 0.78856462]


#### Variance

Similarly to the MVP, variance $\sigma_{\mu}^2$ is calculated with: 
$$
\sigma_{\mu}^2 = \mathbf{w}_{\mu} \ \cdot\ \mathbf{C}\ \cdot\ \mathbf{w}_{\mu}^{\mathbf{T}}
$$

In [10]:
ret_cons_variance = ret_cons_weights @ cov_mat @ ret_cons_weights.T
print(ret_cons_variance)

0.008498808814770695


#### Confirming Return

We will also check that this portfolio returns the required amount with

$$
\mu_\mu = \mathbf{w}_\mu \ \cdot \ \mu
$$

** DOUBLE CHECK ^

In [22]:
ret_cons_return = ret_cons_weights @ exp_returns
print(ret_cons_return)

0.09999999999999999


### Market Portfolio for a given Risk-Free Rate

Choose risk-free rate $0<R<\mu_{\textnormal{MVP}}$, and find the corresponding market portfolio. We can do this by finding the weights of the portfolio

$$
W_m = \frac{(m-Ru)\textnormal{C}^{-1}}{(m-Ru)\textnormal{C}^{-1}u^{\mathbf{T}}}
$$



In [32]:
R = 0.02
Ru = R * u
print(Ru)

m_Ru = exp_returns - Ru
print(m_Ru)

m_Ru_C_inv = m_Ru @ cov_mat_inv
print(m_Ru_C_inv)

m_Ru_C_inv_uT = m_Ru_C_inv @ u.T
print(m_Ru_C_inv_uT)

# Therefore

given_rfr_weights = []

for i in m_Ru_C_inv:
    given_rfr_weights.append(i / m_Ru_C_inv_uT)

print("\n", "weights: ", given_rfr_weights)
print("(negative weights indicate a short position. Total = 1)")


[0.02 0.02 0.02]
[ 0.13 -0.12  0.08]
[ 4.21875    -6.66666667  5.3125    ]
2.8645833333333313

 weights:  [1.4727272727272738, -2.327272727272729, 1.8545454545454552]
(negative weights indicate a short position. Total = 1)


# Beta factor, expected return, risk premium...